In [ ]:
import requests
import math
from collections import Counter
from string import punctuation as enPunc #處理英文標點
!pip install zhon
from zhon.hanzi import punctuation as zhPunc #處理中文標點
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

import pandas as pd
# from elasticsearch import Elasticsearch
from bs4  import BeautifulSoup
!pip install -U LAC
from LAC import LAC
! pip install OpenCC
from opencc import OpenCC

!pip install -U hanlp[full]
import hanlp
import pickle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
## 偵測語言
# !pip install langdetect
# from langdetect import detect

## Part 1. Test data pre-processing

### 1.1 Import test data
- 因為colab在解析時出現很多亂碼，所以直接從本機把解析後的list存成pkl檔，再匯入

In [ ]:
## load data from local file
path = "/content/drive/MyDrive/Colab Notebooks/"
file_name = path+"htmlText.pkl"
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

html_text = loaded_list

In [ ]:
## 振華數據1
search_word = '振華數據'
# google_search_rs = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ner_sample.xlsx').url.to_list()

In [ ]:
html_text[0]

In [ ]:
## colab couldn't work
html_text = []
for idx, link in enumerate(google_search_rs):
    # print(idx+1, link)
    ## 用get下載普通網頁 (把每個連結的資料下載下來，並自訂表頭headers)
    resp = requests.get(link, headers={'user-agent': 
                                       'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
    try:
        resp_txt = resp.content.decode('utf-8')
    except Exception as e:
        print(e)
        resp_txt = resp.text      
    html_text.append(BeautifulSoup(resp_txt).text) ## BeautifulSoup解析

### 1.2 Clean test data 挑出文本
- 刪除特定符號 ['\n', '\r', '\t', '\xa0']
- 繁轉簡
- 匯入stop words檔案 (Biadu stop words)

In [7]:
def multi_repl(input_str, repl_dict):
    for k in repl_dict:
        input_str = input_str.replace(k, repl_dict[k])  ## 把stop words改為空白
    return input_str

#trim_dict
trim = ['\n', '\r', '\t', '\xa0'] ## + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
trim_dict = {pat:'' for pat in trim}

# 繁轉簡
cc = OpenCC('tw2sp')
temp = [cc.convert(texts) for texts in html_text]

In [8]:
## Load stop words
f = open(path+'baidu_stopwords.txt','r')
k = f.readlines()
f.close()
stopwords = [lines.strip('\n') for lines in k]
stopwords.append(cc.convert(search_word))  ## 加入搜尋詞，避免影響關鍵字萃取結果
stopwords.append(cc.convert('振華'))
trim_dict2 = {pat:'' for pat in stopwords}

In [ ]:
# trim_text = [multi_repl(texts,trim_dict) for texts in temp]
# trim_text = [multi_repl(texts,trim_dict2) for texts in trim_text]
# trim_text

### 1.3 Word Segmentation /w NER
- LAC (中文分詞、詞性標註)
- HanLP (多語言分詞)
  - hanlp.pretrained.ALL 查詢所有內建model
  - hanlp.pretrained.~.ALL 可用dir找

##### 1.3.1 LAC NER

In [9]:
# LAC model with ner
lac = LAC(mode='lac')
lac.load_customization('/content/drive/MyDrive/Colab Notebooks/custom_dict.txt')
ner_model = lac.run(temp)

In [10]:
ner_pair = []  # 分詞及標注結果
pos_t = []
POS = []
filtered_ner_word = []
filtered_ner = []  # 分詞及特定ner標注結果 PER, ORG, TITLE
for idx, (word, ner) in enumerate(ner_model):
  temp = []
  pos_t = []
  for num in range(len(word)):
    if word[num] not in stopwords:
      word[num] = multi_repl(word[num],trim_dict)  # remove stop words
      ner_pair.append((word[num],ner[num]))
      if ner[num] == 'PER' or ner[num] == 'ORG' or ner[num] == 'TITLE':
        if word[num] != '':
          pos_t.append(word[num])
          filtered_ner.append((word[num],ner[num],idx+1))
          temp.append(word[num])
  filtered_ner_word.append(temp)
  POS.append(pos_t)

In [11]:
filNER = pd.DataFrame(filtered_ner,columns=['words','ner','category'])

In [23]:
filNER[filNER.category == 6]

,words,ner,category
219,深圳,ORG,6
220,深圳,ORG,6
221,深圳,ORG,6
222,信息技术有限公司,ORG,6
223,深圳,ORG,6
224,信息技术有限公司,ORG,6
225,拓尔思,PER,6
226,华融公司,ORG,6
227,中共中央宣传部,ORG,6
228,中译语通科技股份有限公司,ORG,6


In [ ]:
filtered_ner_word[0][:10] #特定ner結果 只有文字  用於詞頻統計，計算每篇文章

['深圳',
 '信息技术有限公司',
 '深圳',
 '信息技术有限公司',
 '深圳',
 '信息技术有限公司',
 'revival',
 '南山软...  ',
 '振华',
 '深圳']

##### 1.3.2 HanLP NER
- NS: LOC, NT: ORG, NR: PER  
- 結果好像不太理想

In [ ]:
recognizer = hanlp.load(hanlp.pretrained.ner.MSRA_NER_BERT_BASE_ZH)
ner_list = []
for each in trim_text:
  ner_list.append(recognizer(each))  ## NS: LOC, NT: ORG, NR: PER 

In [ ]:
ner_list[:3] 

[[('深圳振华数据信息技术有限公司', 'NT', 7, 21)],
 [('振华数据网站', 'NT', 0, 6), ('王雪峰', 'NR', 10, 13)],
 [('振华', 'NT', 2, 4),
  ('黎智英', 'NR', 14, 17),
  ('台', 'NS', 18, 19),
  ('苹果新闻网', 'NT', 36, 41),
  ('苹果日报', 'NT', 44, 48),
  ('台', 'NS', 49, 50),
  ('湾', 'NT', 50, 51),
  ('香', 'NS', 51, 52),
  ('北美', 'NT', 53, 55),
  ('台', 'NS', 55, 56),
  ('湾', 'NT', 56, 57),
  ('香', 'NS', 57, 58),
  ('北美', 'NT', 59, 61)]]

In [ ]:
for words in ner_list:
  if words != []:
    print(words)

[('深圳振华数据信息技术有限公司', 'NT', 7, 21)]
[('振华数据网站', 'NT', 0, 6), ('王雪峰', 'NR', 10, 13)]
[('振华', 'NT', 2, 4), ('黎智英', 'NR', 14, 17), ('台', 'NS', 18, 19), ('苹果新闻网', 'NT', 36, 41), ('苹果日报', 'NT', 44, 48), ('台', 'NS', 49, 50), ('湾', 'NT', 50, 51), ('香', 'NS', 51, 52), ('北美', 'NT', 53, 55), ('台', 'NS', 55, 56), ('湾', 'NT', 56, 57), ('香', 'NS', 57, 58), ('北美', 'NT', 59, 61)]
[('振华数据CEO', 'NT', 0, 7), ('川普', 'NR', 8, 10), ('深圳振华', 'NT', 28, 32), ('万', 'NT', 38, 39), ('元', 'NT', 51, 52), ('大陆', 'NS', 57, 59), ('美国', 'NS', 59, 61), ('香港', 'NS', 61, 63), ('台湾', 'NS', 65, 67), ('活健', 'NT', 87, 89), ('纽约', 'NS', 93, 95), ('旧金山', 'NS', 95, 98), ('洛杉矶', 'NS', 98, 101), ('华府', 'NS', 101, 103), ('多伦多', 'NS', 103, 106), ('温哥华', 'NS', 106, 109), ('澳洲', 'NS', 109, 111), ('欧洲', 'NS', 111, 113), ('澳大利亚新闻网', 'NT', 115, 122)]
[('振华数据CEO', 'NT', 0, 7), ('川普', 'NR', 8, 10), ('深圳振华', 'NT', 28, 32), ('万', 'NT', 38, 39), ('元', 'NT', 51, 52), ('大陆', 'NS', 57, 59), ('美国', 'NS', 59, 61), ('香港', 'NS', 61, 63), ('台湾', 'NS',

##### 1.3.3 LAC 分詞器 (without ner tagging)
- 可能用於BiLSTM CRF model

In [ ]:
# LAC model only segment
lac1 = LAC(mode='seg')
seg = lac1.run(temp1)

### [Option] 1.4 Ignore stop words
- already load stop words (1.2)
- LAC 純分詞
- HanLP 純分詞
- 用於深度學習BiLSTM_CRF模型


In [ ]:
## HanLP 純分詞
tokenizer = hanlp.load('PKU_NAME_MERGED_SIX_MONTHS_CONVSEG') ## 選擇分詞器 
token = tokenizer(trim_text)
final_token = []
final_token = [[wd for wd in each if wd[0] != ' '] for each in token]
# final_token

In [ ]:
final_text = [[tok for tok in eachrow if tok not in stopwords] for eachrow in final_token]
# final_text

In [ ]:
# save final_text for NER
sample_list = final_text
file_name = path+"finalText.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

## Part 2. Ner modelling (TBC)

### 2.1 Sequence tagging - NER
  - use LAC ner model (1.3.1)
  - use HanLP ner model (1.3.2)
  - use BILSTM+CRF pre-trained model
  - use self training model

## Part 3. Feature Extraction
- term frequency
- TF-IDF
- Text Rank
- Word Embedding

##### 3.1 data from LAC model

In [ ]:
## 詞頻 已刪除搜尋詞、使用LAC標註、特定ner： 計算每篇文章詞頻
each_cnt = []
each_cnt = [Counter(row) for row in filtered_ner_word]

In [ ]:
## view 10 most common words in each content
for idx, each in enumerate(each_cnt):
    print(idx+1,each.most_common(10))

1 [('深圳', 11), ('信息技术有限公司', 7), ('国务院', 5), ('王雪峰', 2), ('天津市振华众研科技', 2), ('李晓宁', 2), ('冯光春', 2), ('南山软...  ', 1), ('总经理', 1), ('赵红飞', 1)]
2 [('王雪峰', 5), ('中国电子信息产业集团', 2), ('IBM', 2), ('CETC', 2), ('澳洲广播公司', 1), ('公司', 1), ('国际商业机器公司', 1), ('中国振华电子集团', 1), ('悉尼科技大学', 1), ('振华', 1)]
3 [('黎智英', 5), ('周鹏', 5), ('王雪峰', 4), ('罗宾森', 4), ('深圳', 3), ('振华公司', 3), ('壹传媒', 2), ('中国政府', 2), ('信息技术有限公司', 2), ('岑子杰', 2)]
4 [('伊万卡', 6), ('王雪峰', 5), ('中共', 5), ('川普', 3), ('深圳', 3), ('信息技术有限公司', 3), ('总统', 2), ('川普长女成目标中共', 1), ('Jim Watson/Getty Images', 1), ('李睿悉尼', 1)]
5 [('伊万卡', 6), ('王雪峰', 5), ('中共', 5), ('川普', 3), ('深圳', 3), ('信息技术有限公司', 3), ('总统', 2), ('川普长女成目标中共', 1), ('Jim Watson/Getty Images', 1), ('李睿悉尼', 1)]
6 [('深圳', 5), ('维基媒体基金会', 3), ('信息技术有限公司', 2), ('拓尔思', 1), ('华融公司', 1), ('中共中央宣传部', 1), ('中译语通科技股份有限公司', 1), ('北京大学', 1), ('鲍尔丁', 1), ('中国人民解放军', 1)]
7 []
8 [('澳洲广播公司', 4), ('克里斯托弗·巴尔德', 4), ('BBC', 4), ('约翰逊', 3), ('波特', 3), ('越南富布赖特大学', 2), ('行政总裁', 2), ('王雪峰', 2), ('罗杰·卡尔', 2), ('总统

In [ ]:
l = []
df = pd.DataFrame()
temp = pd.DataFrame()
for idx, each in enumerate(each_cnt):
  a = [idx+1]*len(each.most_common(20))
  df = df.append(pd.DataFrame(each.most_common(20),columns=['words','freq']))
  temp = temp.append(pd.DataFrame(a, columns=['cat.']))
df = pd.concat([df,temp],axis=1)

In [ ]:
# Counter(df['words']).most_common()
df

,words,freq,cat.
0,深圳,11,1
1,信息技术有限公司,7,1
2,国务院,5,1
3,王雪峰,2,1
4,天津市振华众研科技,2,1
...,...,...,...
15,北京大学,2,50
16,汇丰商学院,2,50
17,华为,2,50
18,公司,2,50


In [ ]:
common_df = df[df['freq']>=2]
# common_df
Counter(common_df['words']).most_common(20)

[('中共', 19),
 ('王雪峰', 15),
 ('深圳', 13),
 ('鲍尔丁', 11),
 ('信息技术有限公司', 10),
 ('中国政府', 10),
 ('振华公司', 8),
 ('总统', 8),
 ('鲍丁', 7),
 ('马英九', 7),
 ('波特', 6),
 ('公司', 6),
 ('北京大学', 5),
 ('宋楚瑜', 5),
 ('川普', 4),
 ('习近平', 4),
 ('自由亚洲电台', 4),
 ('沈伯洋', 4),
 ('澳洲广播公司', 3),
 ('约翰逊', 3)]

In [ ]:
from gensim.models.word2vec import Word2Vec

2021-05-12 08:31:06,292 - INFO - 'pattern' package not found; tag filters are not available for English


In [ ]:
# 斷詞後存成dataframe,以篇數為基準
w2v = pd.DataFrame(columns=['texts','cat.'])
for idx, text in enumerate(POS):
    w2v = w2v.append({'texts':text,'cat.':idx},ignore_index=True)
w2v.head()

,texts,cat.
0,"[深圳, 振华数据, 信息技术有限公司, 深圳, 振华数据, 信息技术有限公司, 深圳, 振...",0
1,"[振华数据, 王雪峰, 振华数据, 王雪峰, 振华数据, 中国电子信息产业集团, 澳洲广播公...",1
2,"[振华数据, 黎智英, 壹会员, 振华数据, 黎智英, 周鹏, 深圳, 振华数据, 振华, ...",2
3,"[振华数据, 振华数据, 川普长女成目标中共, 川普, 伊万卡, 川普, 总统, 伊万卡, ...",3
4,"[振华数据, 振华数据, 川普长女成目标中共, 川普, 伊万卡, 川普, 总统, 伊万卡, ...",4


In [ ]:
corpus = w2v.texts.sample(frac=1)
model = Word2Vec(corpus) #陽春型 - 總共1602個詞，每個詞有100維

In [ ]:
## 檢驗訓練好的詞向量模型
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [ ]:
# w = cc.convert(search_word)
most_similar(model, ['振华数据','王雪峰','中国人民解放军', '国务院','澳大利亚广播公司'])

,振华数据,cos,王雪峰,cos,中国人民解放军,cos,国务院,cos,澳大利亚广播公司,cos
0,深圳,0.989674,振华数据,0.984609,深圳,0.973230,振华数据,0.937765,振华数据,0.973241
1,总统,0.988857,总统,0.980739,振华数据,0.973031,总统,0.937591,总统,0.969940
2,鲍尔丁,0.985231,马英九,0.980139,王雪峰,0.972833,振华,0.935505,深圳,0.969310
3,马英九,0.985213,深圳,0.979474,深圳振华,0.970760,深圳振华,0.934539,王雪峰,0.968379
4,王雪峰,0.984609,深圳振华,0.978085,总统,0.970097,深圳,0.933138,中共,0.967906
5,振华,0.983969,伊万卡,0.976078,中共,0.969243,华为,0.929820,鲍尔丁,0.967436
6,深圳振华,0.983591,鲍尔丁,0.975363,马英九,0.969096,伊万卡,0.927812,IBM,0.965938
7,中共,0.982451,振华,0.975359,鲍尔丁,0.967068,IBM,0.925173,振华,0.965504
8,Robert,0.982184,中共,0.974161,Robert,0.966719,鲍尔丁,0.924722,伊万卡,0.964809
9,伊万卡,0.980289,中国人民解放军,0.972833,中国政府,0.964265,澳大利亚广播公司,0.924571,北京大学,0.964604


In [ ]:
cat1 = filNER[filNER.category == 3]
Counter(filNER['words']).most_common()

TypeError: ignored